[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/feiglab/cocomo/blob/main/cocomo.ipynb)

In [10]:
## FIRST activate GPU usage by: Runtime -> Change run time type -> GPU
## check if openmm is in Colab, otherwise install it
import sys
import os
import subprocess

pkgs = [ i.split('==')[0].lower() for i in subprocess.getoutput('pip list --format=freeze').splitlines() ]
if 'openmm' in pkgs:
  pass
else:
  print('No OpenMM available.\nPlease run the next cell to install it.')

No OpenMM available.
Please run the next cell to install it.


In [64]:
## Installing OpenMM via conda, if already done
## run the python command only
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local
!conda config --set always_yes yes
!conda config --add channels conda-forge
!conda create -n openmm python=3.7 cudatoolkit=10.0 numpy nglview openmm openmoltools openmmforcefields
sys.path.append('/usr/local/envs/openmm/lib/python3.7/site-packages')

In [12]:
## Resources and files

resources = 'CUDA' # for using solely CPUs set resources = 'CPU'

!if [ ! -f cocomo.pdb ] ; then wget https://raw.githubusercontent.com/feiglab/cocomo/main/cocomo.pdb ; fi
pdb_file = 'cocomo.pdb'

In [14]:
## Load libraries

from __future__ import print_function
import numpy as np

from openmm.unit import *
from openmm import *
from openmm.app import *

from google.colab import output
output.enable_custom_widget_manager()
import mdtraj as md
import nglview as nv

In [15]:
## COCOMO Force Field parameters

kbond = 4184
l0_pro = 0.38
l0_rna = 0.5
theta0 = 180
kangle_pro = 4.184 
kangle_rna = 5.021
eps_polar = 0.40
eps_nopol = 0.41
eps_rna = 0.41
cationpi_propro = 0.30
cationpi_prorna = 0.20
kappa = 1.00

ff_param = {'ALA': {'mass':  71.079, 'charge':  0.0, 'radius': 0.2845, 'epsilon': 0.41, 'azero': 0.00}, 
            'ARG': {'mass': 157.197, 'charge':  1.0, 'radius': 0.3567, 'epsilon': 0.40, 'azero': 0.05}, 
            'ASN': {'mass': 114.104, 'charge':  0.0, 'radius': 0.3150, 'epsilon': 0.40, 'azero': 0.05}, 
            'ASP': {'mass': 114.080, 'charge': -1.0, 'radius': 0.3114, 'epsilon': 0.40, 'azero': 0.05}, 
            'CYS': {'mass': 103.139, 'charge':  0.0, 'radius': 0.3024, 'epsilon': 0.40, 'azero': 0.05}, 
            'GLN': {'mass': 128.131, 'charge':  0.0, 'radius': 0.3311, 'epsilon': 0.40, 'azero': 0.05}, 
            'GLU': {'mass': 128.107, 'charge': -1.0, 'radius': 0.3279, 'epsilon': 0.40, 'azero': 0.05}, 
            'GLY': {'mass':  57.052, 'charge':  0.0, 'radius': 0.2617, 'epsilon': 0.41, 'azero': 0.00}, 
            'HIS': {'mass': 137.142, 'charge':  0.0, 'radius': 0.3338, 'epsilon': 0.40, 'azero': 0.05}, 
            'ILE': {'mass': 113.160, 'charge':  0.0, 'radius': 0.3360, 'epsilon': 0.41, 'azero': 0.00}, 
            'LEU': {'mass': 113.160, 'charge':  0.0, 'radius': 0.3363, 'epsilon': 0.41, 'azero': 0.00}, 
            'LYS': {'mass': 129.183, 'charge':  1.0, 'radius': 0.3439, 'epsilon': 0.40, 'azero': 0.05}, 
            'MET': {'mass': 131.193, 'charge':  0.0, 'radius': 0.3381, 'epsilon': 0.41, 'azero': 0.00}, 
            'PHE': {'mass': 147.177, 'charge':  0.0, 'radius': 0.3556, 'epsilon': 0.41, 'azero': 0.00}, 
            'PRO': {'mass':  98.125, 'charge':  0.0, 'radius': 0.3187, 'epsilon': 0.41, 'azero': 0.00}, 
            'SER': {'mass':  87.078, 'charge':  0.0, 'radius': 0.2927, 'epsilon': 0.40, 'azero': 0.05}, 
            'THR': {'mass': 101.105, 'charge':  0.0, 'radius': 0.3108, 'epsilon': 0.40, 'azero': 0.05}, 
            'TRP': {'mass': 186.214, 'charge':  0.0, 'radius': 0.3754, 'epsilon': 0.41, 'azero': 0.00}, 
            'TYR': {'mass': 163.176, 'charge':  0.0, 'radius': 0.3611, 'epsilon': 0.41, 'azero': 0.00}, 
            'VAL': {'mass':  99.133, 'charge':  0.0, 'radius': 0.3205, 'epsilon': 0.41, 'azero': 0.00}, 
            'ADE': {'mass': 315.697, 'charge': -1.0, 'radius': 0.4220, 'epsilon': 0.41, 'azero': 0.05}, 
            'CYT': {'mass': 305.200, 'charge': -1.0, 'radius': 0.4110, 'epsilon': 0.41, 'azero': 0.05}, 
            'GUA': {'mass': 345.200, 'charge': -1.0, 'radius': 0.4255, 'epsilon': 0.41, 'azero': 0.05}, 
            'URA': {'mass': 305.162, 'charge': -1.0, 'radius': 0.4090, 'epsilon': 0.41, 'azero': 0.05}} 

In [27]:
## Visualize initial coordinates

traj = md.load(pdb_file)
view = nv.show_mdtraj(traj, default_representation=False)
view.background = 'transparent'
view.camera = 'orthographic'
view.add_representation('axes', showAxes=False, showBox=True, radius=3.0, color='black')
view.add_representation('point', selection='A', color='red', pointSize=3, useTexture=True, sizeAttenuation=False)
view.add_representation('point', selection='protein', color='blue', pointSize=3, useTexture=True, sizeAttenuation=False)
view.center(selection='all')
print('\033[94m' + 'Protein' '\n' '\033[91m' + 'RNA' )
view

In [17]:
## Read atom list and coordinates from PDB file

def read_pdb (filename):
    atoms = []
    with open(filename, 'r') as f:
        for line in f:
            line = line.split()
            if line[0] == 'ATOM':
                atoms.append([line[2], line[3], line[10]])
    atoms = np.array(atoms, dtype='object')
    return atoms

positions = PDBFile(pdb_file).positions
atom_list = read_pdb(pdb_file)
segnames = [atom_list[i,2] != atom_list[i+1,2] for i in range(-1,atom_list.shape[0]-1)]
segnames = atom_list[segnames,2]

In [18]:
## Generate topolgy
top = topology.Topology()
for seg in segnames:
    chain = top.addChain(seg)
    for atm in atom_list:
        if atm[2] == seg:
            residue = top.addResidue(atm[1], chain)
            top.addAtom(atm[0], element=element.carbon, residue=residue)
            #
    atom = [i for i in chain.atoms()]
    for i in range(len(atom)-1):
        top.addBond(atom[i],atom[i+1])

BOX = 100.
a = Quantity(np.zeros([3]), nanometers)
a[0] = BOX * nanometers
b = unit.Quantity(np.zeros([3]), nanometers)
b[1] = BOX * nanometers
c = unit.Quantity(np.zeros([3]), nanometers)
c[2] = BOX * nanometers
box = (a,b,c)
top.setPeriodicBoxVectors(box)

In [19]:
## Build system
system = openmm.System()

# add Particles to the system
for i in atom_list[:,1]: system.addParticle(ff_param[i]['mass']*unit.amu) 

# set BOX vectors to the system 
system.setDefaultPeriodicBoxVectors(a, b, c)

# Set energy switching function on or use cutoff only
# Switching implementation following Steinbach & Brooks paper
# equation (10), https://onlinelibrary.wiley.com/doi/epdf/10.1002/jcc.540150702
# this switching implementation slows simulation by ~20%
SWITCHING = True 

if SWITCHING == True:
    R_ON  = 2.9
    R_OFF = 3.1
else:
    R_OFF = 3.0

In [20]:
## Add COCOMO forces

# bond force
f_bond = openmm.HarmonicBondForce()
for bond in top.bonds():
    if bond[0].residue.name in ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'ILE','LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL']:
        f_bond.addBond(bond[0].index, bond[1].index, l0_pro*nanometer, kbond*kilojoules_per_mole/(nanometer**2))
    elif bond[0].residue.name in ['ADE', 'CYT', 'GUA', 'URA']:
        f_bond.addBond(bond[0].index, bond[1].index, l0_rna*nanometer, kbond*kilojoules_per_mole/(nanometer**2))

system.addForce(f_bond)

# angle force
f_angle = openmm.HarmonicAngleForce()
for atoms in [[i for i in top.atoms() if i.residue.chain.id == seg] for seg in segnames]:
    if atoms[0].residue.name in ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'ILE','LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL']:
        for i in range(len(atoms)-2):
            f_angle.addAngle(atoms[i].index, atoms[i+1].index, atoms[i+2].index, theta0*degrees, kangle_pro*kilojoule_per_mole/(radian**2))
    elif atoms[0].residue.name in ['ADE', 'CYT', 'GUA', 'URA']:
        for i in range(len(atoms)-2):
            f_angle.addAngle(atoms[i].index, atoms[i+1].index, atoms[i+2].index, theta0*degrees, kangle_rna*kilojoule_per_mole/(radian**2))

system.addForce(f_angle)

# openMM Center of Mass Motion Remover
system.addForce(openmm.CMMotionRemover())

# electrostactic
k0 = kappa*nanometer

if SWITCHING == True:
    equation1  = "select( step(r_on-r), longrange+sdel, switch ); "
    equation1 += "longrange = (A+Z)*exp(-r/K0)/r; "
    equation1 += "sdel = sk*((1/r_on)^3-1/(r_off)^3)^2 - (A+Z)/r_on*exp(-r_on/K0); "
    equation1 += "switch = sk*((1/r)^3-1/(r_off)^3)^2; "
    equation1 += "sk = -longrange_deriv_Ron/switch_deriv_Ron; "
    equation1 += "longrange_deriv_Ron = -1*(A+Z)*exp(-r_on/K0)/r_on*(1/K0+1/r_on); "
    equation1 += "switch_deriv_Ron = 6*(1/r_on^3-1/r_off^3)*1/r_on^4; "
    equation1 += "A=A1*A2; "
    equation1 += "Z=Z1+Z2 "
else:
    equation1  = "(A+Z)/r*exp(-r/K0); "
    equation1 += "A=A1*A2; "
    equation1 += "Z=Z1+Z2"

force1 = CustomNonbondedForce(equation1)
force1.addGlobalParameter("K0", k0)
force1.addPerParticleParameter("A")
force1.addPerParticleParameter("Z")
force1.setNonbondedMethod(CustomNonbondedForce.CutoffPeriodic)
force1.setCutoffDistance(R_OFF*nanometer)
if SWITCHING == True:
    force1.addGlobalParameter("r_on", R_ON)
    force1.addGlobalParameter("r_off", R_OFF)

for atom in top.atoms():
    force1.addParticle([(np.sqrt(0.75*np.abs(ff_param[atom.residue.name]['charge']))*np.sign(ff_param[atom.residue.name]['charge']))*nanometer*kilojoule/mole, ff_param[atom.residue.name]['azero'] *(nanometer*kilojoule/mole)**(1/2)])

force1.createExclusionsFromBonds([(i[0].index, i[1].index) for i in top.bonds()], 1)
system.addForce(force1)

# short range
if SWITCHING == True:
    equation2  = "select( step(r_on-r), shortrange+sdel, switch ); "
    equation2 += "shortrange = 4*epsilon*((sigma/r)^10-(sigma/r)^5); "
    equation2 += "sdel = sk*((1/r_on)^3-1/(r_off)^3)^2 - 4*epsilon*((sigma/r_on)^10-(sigma/r_on)^5); "
    equation2 += "switch = sk*((1/r)^3-1/(r_off)^3)^2; "
    equation2 += "sk = -shortrange_deriv_Ron/switch_deriv_Ron; "
    equation2 += "shortrange_deriv_Ron = 4*epsilon*(-10*(sigma/r_on)^11*1/r_on+5*(sigma/r_on)^5*1/r_on ); "
    equation2 += "switch_deriv_Ron = 6*(1/r_on^3-1/r_off^3)*1/r_on^4; "
    equation2 += "sigma=0.5*(sigma1+sigma2); "
    equation2 += "epsilon=sqrt(epsilon1*epsilon2)"
else:
    equation2  = "4*epsilon*((sigma/r)^10-(sigma/r)^5); "
    equation2 += "sigma=0.5*(sigma1+sigma2); "
    equation2 += "epsilon=sqrt(epsilon1*epsilon2)"

force2 = CustomNonbondedForce(equation2)
force2.addPerParticleParameter("sigma")
force2.addPerParticleParameter("epsilon")
force2.setNonbondedMethod(CustomNonbondedForce.CutoffPeriodic)
force2.setCutoffDistance(R_OFF*nanometer)
if SWITCHING == True:
    force2.addGlobalParameter("r_on", R_ON)
    force2.addGlobalParameter("r_off", R_OFF)


for atom in top.atoms():
    force2.addParticle([ff_param[atom.residue.name]['radius']*2*2**(-1/6)*nanometer, ff_param[atom.residue.name]['epsilon']*kilojoule/mole])

force2.createExclusionsFromBonds([(i[0].index, i[1].index) for i in top.bonds()], 1)
system.addForce(force2)
    
# protein - protein cation pi 
if any(True for i in ['ARG', 'LYS'] if i in [atom.residue.name for atom in top.atoms()]) and any(True for i in ['PHE', 'TRP', 'TYR'] if i in [atom.residue.name for atom in top.atoms()]) :
    force3 = CustomNonbondedForce(equation2)
    force3.addPerParticleParameter("sigma")
    force3.addPerParticleParameter("epsilon")
    force3.setNonbondedMethod(CustomNonbondedForce.CutoffPeriodic)
    force3.setCutoffDistance(R_OFF*nanometer)
    if SWITCHING == True:
        force3.addGlobalParameter("r_on", R_ON)
        force3.addGlobalParameter("r_off", R_OFF)
    for atom in top.atoms():
        force3.addParticle([ff_param[atom.residue.name]['radius']*2*2**(-1/6)*nanometer, cationpi_propro*kilojoule/mole])
        #
    force3.createExclusionsFromBonds([(i[0].index, i[1].index) for i in top.bonds()], 1)
    arg_lys  = [atom.index for atom in top.atoms() if atom.residue.name in ['ARG', 'LYS']]
    aromatic = [atom.index for atom in top.atoms() if atom.residue.name in ['PHE', 'TRP', 'TYR']]
    force3.addInteractionGroup(arg_lys, aromatic)
    system.addForce(force3)
    
# protein - rna cation pi
if any(True for i in ['ARG', 'LYS'] if i in [atom.residue.name for atom in top.atoms()]) and any(True for i in ['ADE', 'CYT', 'GUA', 'URA'] if i in [atom.residue.name for atom in top.atoms() ]) :
    force4 = CustomNonbondedForce(equation2)
    force4.addPerParticleParameter("sigma")
    force4.addPerParticleParameter("epsilon")
    force4.setNonbondedMethod(CustomNonbondedForce.CutoffPeriodic)
    force4.setCutoffDistance(R_OFF*nanometer)
    if SWITCHING == True:
        force4.addGlobalParameter("r_on", R_ON)
        force4.addGlobalParameter("r_off", R_OFF)
    for atom in top.atoms():
        force4.addParticle([ff_param[atom.residue.name]['radius']*2*2**(-1/6)*nanometer, cationpi_prorna*kilojoule/mole])
        #
    force4.createExclusionsFromBonds([(i[0].index, i[1].index) for i in top.bonds()], 1)
    arg_lys  = [atom.index for atom in top.atoms() if atom.residue.name in ['ARG', 'LYS']]
    nucleic = [atom.index for atom in top.atoms() if atom.residue.name in ['ADE', 'CYT', 'GUA', 'URA']]
    force4.addInteractionGroup(arg_lys, nucleic)
    system.addForce(force4)

In [23]:
# Build simulation context
integrator = LangevinIntegrator(298*kelvin, 0.01/picosecond, 0.01*picoseconds)

# Set platform
platform = Platform.getPlatformByName(resources)
if resources == 'CUDA' :
    prop = dict(CudaPrecision='mixed')

if resources == 'CUDA' : simulation = Simulation(top, system, integrator, platform, prop)
if resources == 'CPU'  : simulation = Simulation(top, system, integrator, platform)

simulation.context.setPositions(positions)

print("\nInitial system energy")
print(simulation.context.getState(getEnergy=True).getPotentialEnergy())

# minimization
mini_nstep = 5000
print("\nEnergy minimization: %s steps" % mini_nstep)
simulation.minimizeEnergy(tolerance=100.0*kilojoule/(nanometer*mole), maxIterations=mini_nstep)
print(simulation.context.getState(getEnergy=True).getPotentialEnergy())

# generate velocities
simulation.context.setVelocitiesToTemperature(298*kelvin, 870516298)

# run simulation
nstep  = 500#0000  # number of steps to run, 100 ns, ~10 min running with GPU
nstout = 50#000    # Writing output frequency (steps), every 1 ns
nstdcd = 50#000    # Writing coords frequency (steps), every 1 ns
odcd = 'trajectory.dcd'
olog = 'trajectory.log'
simulation.reporters.append(DCDReporter(odcd, nstdcd))
simulation.reporters.append(StateDataReporter(olog, nstout, step=True, time=True, potentialEnergy=True, kineticEnergy=True, totalEnergy=True, temperature=True, volume=True, density=True, progress=True, remainingTime=True, speed=True, totalSteps=nstep, separator='\t'))

print("\nMD run: %s steps" % nstep)
simulation.step(nstep)
print(simulation.context.getState(getEnergy=True).getPotentialEnergy())


Initial system energy
17842.388734857836 kJ/mol

Energy minimization: 5000 steps
10950.310625098897 kJ/mol

MD run: 500 steps
15685.778364480473 kJ/mol


In [26]:
# Visualize final coordinates

traj = md.load('trajectory.dcd', top=pdb_file)[-1]
view = nv.show_mdtraj(traj, default_representation=False)
view.background = 'transparent'
view.camera = 'orthographic'
view.add_representation('axes', showAxes=False, showBox=True, radius=3.0, color='black')
view.add_representation('point', selection='A', color='red', pointSize=3, useTexture=True, sizeAttenuation=False)
view.add_representation('point', selection='protein', color='blue', pointSize=3, useTexture=True, sizeAttenuation=False)
view.center(selection='all')
print('\033[94m' + 'Protein' '\n' '\033[91m' + 'RNA' )
view